In [1]:
from core.agent import agent_executor, agent_llm
from core.CustomTools import fetch_and_rerank

from langchain.agents.openai_functions_agent.agent_token_buffer_memory import (
    AgentTokenBufferMemory,
)


from datasets import Dataset

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

import regex as re
import glob

from core.ragas import RagasCallbackHandler

# ---

#agent_memory = AgentTokenBufferMemory(llm=agent_llm)
callback_handler = RagasCallbackHandler()
def query_agent(prompt):
   response = agent_executor(
            {"input": prompt, "history": []},                        
            callbacks=[callback_handler],
            include_run_info=True,
        )
   return response



In [2]:
# source data

filenames = []
for _file in glob.glob("ragas_data/*.txt"):
    filenames.append(_file)

questions = []
ground_truths = []

DELIM = "__###__"
for name in filenames:
    file_content = None
    with open(name) as f:                
        file_content = f.read()

    content = file_content.split(DELIM)
    questions.append(content[0])
    truths = []
    truths.append(content[1])
    ground_truths.append(truths)

print("Questions:", len(questions))
print("Grounded Truths:", len(ground_truths))
    

Questions: 3
Grounded Truths: 3


In [3]:
answers = []
contexts = []

# Inference
for query in questions:
  
  # retrieve an answer
  response = query_agent(query)
  answers.append(response["output"])

  inputs, outputs = callback_handler.clear_collected_outputs()
  print("Input: ", len(inputs))
  print("Outputs: ", len(outputs))

  results = []  
  [results.append(doc) for doc in outputs]          
  contexts.append(results)  

print("Answer len:", len(contexts))
print("Context len:", len(contexts))





> Entering new AgentExecutor chain...
starting tool ...  file idempotent
Embedding ms:  0.32719993591308594
0.812 :  _next_file-component.html.txt_26
0.805 :  _next_file-component.html.txt_25
0.794 :  _next_azure-files-component.html.txt_11
0.788 :  _next_file-component.html.txt_14
0.781 :  _eips_competing-consumers.html.txt_2
ending tool ... 

> Finished chain.
Input:  1
Outputs:  1


> Entering new AgentExecutor chain...
starting tool ...  event aggregation in Apache Camel
Embedding ms:  0.37224912643432617
0.831 :  _faq_how-do-the-direct-event-seda-and-vm-endpoints-compare.html.txt
0.829 :  _others_cloudevents.html.txt
0.82 :  _eips_aggregate-eip.html.txt_10
0.829 :  _next_timer-component.html.txt_3
0.83 :  _eips_split-eip.html.txt_10
ending tool ... 

> Finished chain.
Input:  1
Outputs:  1


> Entering new AgentExecutor chain...
starting tool ...  file splitting
Embedding ms:  0.4099709987640381
0.783 :  _next_hdfs-component.html.txt_7
0.781 :  _eips_split-eip.html.txt_0
0.772 :

In [4]:
# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()

df.head()

evaluating with [context_precision]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.08s/it]


evaluating with [context_recall]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:56<00:00, 56.05s/it]


evaluating with [faithfulness]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:25<00:00, 25.14s/it]


evaluating with [answer_relevancy]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.58s/it]


,question,answer,contexts,ground_truths,context_precision,context_recall,faithfulness,answer_relevancy
0,"This is my route\n\n from(""sftp://userName:...",To avoid processing the same files in case of ...,"[page_content='""Using a file based idempotent ...",[\nTo avoid processing the same files in case ...,0.0,1.0,0.666667,0.770217
1,I am having an issue while working on the requ...,To listen to the events generated by one route...,"[page_content='""Article source: https://rhaeto...",[\nTo listen to the events of one route by ano...,1.0,1.0,0.400000,0.795870
2,"I want to read a local file, create equally si...",To achieve the desired functionality of readin...,"[page_content='""String\n\n\n\nCamelFileNameCon...","[\nTo achieve this, you can use Apache Camel t...",1.0,NaN,0.200000,0.792767
